In [1]:
import tensorflow as tf

In [2]:
a = tf.constant([1.0, 2.0], name='a')
b = tf.constant([2.0, 3.0], name='b')

result = a + b

In [3]:
sess = tf.Session()
sess.run(result)

array([ 3.,  5.], dtype=float32)

# 3.4.5 完整神经网络样例程序

In [4]:
import tensorflow as tf
from numpy.random import RandomState

batch_size = 8

w1 = tf.Variable(tf.random_normal([2, 3], stddev=1, seed=1))
w2 = tf.Variable(tf.random_normal([3, 1], stddev=1, seed=1))

x = tf.placeholder(tf.float32, shape=(None, 2), name='x-input')
y_ = tf.placeholder(tf.float32, shape=(None, 1), name='y-input')

a = tf.matmul(x, w1)
y = tf.matmul(a, w2)

cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y, 1e-10, 1.0)))
train_step = tf.train.AdamOptimizer(0.001).minimize(cross_entropy)

rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)
Y = [[int(x1 + x2 < 1)] for (x1, x2) in X]

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    print(sess.run(w1))
    print(sess.run(w2))
    
    STEPS = 5000
    for i in range(STEPS):
        start = (i * batch_size) % dataset_size
        end = min(start + batch_size, dataset_size)
        
        sess.run(train_step, feed_dict={x: X[start:end], y_: Y[start:end]})
        if i % 1000 == 0:
            total_cross_entropy = sess.run(cross_entropy, feed_dict={x: X, y_: Y})
            print("After %d training step(s), cross entropy on all data is %g" %
                 (i, total_cross_entropy))
    
    print(sess.run(w1))
    print(sess.run(w2))

[[-0.81131822  1.48459876  0.06532937]
 [-2.44270396  0.0992484   0.59122431]]
[[-0.81131822]
 [ 1.48459876]
 [ 0.06532937]]
After 0 training step(s), cross entropy on all data is 0.0674925
After 1000 training step(s), cross entropy on all data is 0.0163385
After 2000 training step(s), cross entropy on all data is 0.00907547
After 3000 training step(s), cross entropy on all data is 0.00714436
After 4000 training step(s), cross entropy on all data is 0.00578471
[[-1.9618274   2.58235407  1.68203783]
 [-3.4681716   1.06982327  2.11788988]]
[[-1.8247149 ]
 [ 2.68546653]
 [ 1.41819501]]


# 4.2.2 自定义损失函数

In [5]:
import tensorflow as tf
from numpy.random import RandomState

batch_size = 8

x = tf.placeholder(tf.float32, shape=(None, 2), name='x-input')
y_ = tf.placeholder(tf.float32, shape=(None, 1), name='y-input')

w1 = tf.Variable(tf.random_normal([2, 1], stddev=1, seed=1))
y = tf.matmul(x, w1)

loss_less = 10
loss_more = 1

loss = tf.reduce_sum(tf.where(tf.greater(y, y_), (y - y_)*loss_more, (y_ - y)*loss_less))
train_step = tf.train.AdamOptimizer(0.001).minimize(loss)

rdm = RandomState(1)
dataset_size = 128
X = rdm.rand(dataset_size, 2)
Y = [[x1 + x2 + rdm.rand()/10.0] for (x1, x2) in X]

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    STEPS = 5000
    for i in range(STEPS):
        start = (i * batch_size) % dataset_size
        end = min(start + batch_size, dataset_size)
        
        sess.run(train_step, feed_dict={x: X[start:end], y_: Y[start:end]})
        if i % 1000 == 0:
            total_loss = sess.run(loss, feed_dict={x: X, y_: Y})
            print("After %d training step(s), loss on all data is %g" %
                 (i, total_loss))
    
    print(sess.run(w1))

After 0 training step(s), loss on all data is 814.831
After 1000 training step(s), loss on all data is 187.065
After 2000 training step(s), loss on all data is 93.1723
After 3000 training step(s), loss on all data is 62.4251
After 4000 training step(s), loss on all data is 28.8475
[[ 1.07596886]
 [ 1.10207963]]


# 5.1 MNIST 数据处理

In [12]:
from tensorflow.examples.tutorials.mnist import input_data
import os

if os.path.exists("/Users"):
    mnist = input_data.read_data_sets("/Users/colinzuo/work/github/personal_study/"
                                      "tensorflow/book_caicloud/mnist/", one_hot=True)
else:
    mnist = input_data.read_data_sets("D:\\work\\DataAnalysis\\study\\mnist", one_hot=True)

Extracting /Users/colinzuo/work/github/personal_study/tensorflow/book_caicloud/mnist/train-images-idx3-ubyte.gz
Extracting /Users/colinzuo/work/github/personal_study/tensorflow/book_caicloud/mnist/train-labels-idx1-ubyte.gz
Extracting /Users/colinzuo/work/github/personal_study/tensorflow/book_caicloud/mnist/t10k-images-idx3-ubyte.gz
Extracting /Users/colinzuo/work/github/personal_study/tensorflow/book_caicloud/mnist/t10k-labels-idx1-ubyte.gz


In [13]:
print("Training data size: ", mnist.train.num_examples)
print("Validating data size: ", mnist.validation.num_examples)
print("Testing data size: ", mnist.test.num_examples)
print("Example training data: ", mnist.train.images[0])
print("Example training label: ", mnist.train.labels[0])

Training data size:  55000
Validating data size:  5000
Testing data size:  10000
Example training data:  [ 0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.        

In [14]:
batch_size = 100
xs, ys = mnist.train.next_batch(batch_size)
print("X shape: ", xs.shape)
print("Y shape: ", ys.shape)

X shape:  (100, 784)
Y shape:  (100, 10)


# 5.2.1 TensorFlow 训练神经网络

In [16]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

INPUT_NODE = 784
OUTPUT_NODE = 10

LAYER1_NODE = 500
BATCH_SIZE = 100

LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99

REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 30000
MOVING_AVERAGES_DECAY = 0.99

def inference(input_tensor, avg_class, weights1, biases1, weights2, biases2):
    if avg_class == None:
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights1) + biases1)
        
        return tf.matmul(layer1, weights2) + biases2
    else:
        layer1 = tf.nn.relu(tf.matmul(input_tensor, avg_class.average(weights1)) + avg_class.average(biases1))
        
        return tf.matmul(layer1, avg_class.average(weights2)) + avg_class.average(biases2)
    
def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
    
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
    biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))
    
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
    biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))
    
    y = inference(x, None, weights1, biases1, weights2, biases2)
    
    global_step = tf.Variable(0, trainable=False)
    
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGES_DECAY, global_step)
    
    variables_average_op = variable_averages.apply(tf.trainable_variables())
    
    average_y = inference(x, variable_averages, weights1, biases1, weights2, biases2)
    
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    regularization = regularizer(weights1) + regularizer(weights2)
    loss = cross_entropy_mean + regularization
    
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, global_step, 
                                               mnist.train.num_examples / BATCH_SIZE, LEARNING_RATE_DECAY)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    with tf.control_dependencies([train_step, variables_average_op]):
        train_op = tf.no_op(name='train')
        
    correct_prediction = tf.equal(tf.argmax(average_y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
        test_feed = {x: mnist.test.images, y_: mnist.test.labels}
        
        for i in range(TRAINING_STEPS):
            if i % 1000 == 0:
                validate_acc = sess.run(accuracy, feed_dict=validate_feed)
                test_acc = sess.run(accuracy, feed_dict=test_feed)
                print("After %d training step(s), validation accuracy using average model is %g, "
                      "test accuracy using average model is %g" % (i, validate_acc, test_acc))
            
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op, feed_dict={x: xs, y_: ys})
        
        test_acc = sess.run(accuracy, feed_dict=test_feed)
        print("After %d training steps, test accuracy using average model is %g" % (TRAINING_STEPS, test_acc))
        
def main(argv=None):
    import os

    if os.path.exists("/Users"):
        mnist = input_data.read_data_sets("/Users/colinzuo/work/github/personal_study/"
                                          "tensorflow/book_caicloud/mnist/", one_hot=True)
    else:
        mnist = input_data.read_data_sets("D:\\work\\DataAnalysis\\study\\mnist", one_hot=True)
        
    train(mnist)

main()

Extracting /Users/colinzuo/work/github/personal_study/tensorflow/book_caicloud/mnist/train-images-idx3-ubyte.gz
Extracting /Users/colinzuo/work/github/personal_study/tensorflow/book_caicloud/mnist/train-labels-idx1-ubyte.gz
Extracting /Users/colinzuo/work/github/personal_study/tensorflow/book_caicloud/mnist/t10k-images-idx3-ubyte.gz
Extracting /Users/colinzuo/work/github/personal_study/tensorflow/book_caicloud/mnist/t10k-labels-idx1-ubyte.gz
After 0 training step(s), validation accuracy using average model is 0.1008, test accuracy using average model is 0.1115
After 1000 training step(s), validation accuracy using average model is 0.9786, test accuracy using average model is 0.9743
After 2000 training step(s), validation accuracy using average model is 0.9832, test accuracy using average model is 0.9799
After 3000 training step(s), validation accuracy using average model is 0.9812, test accuracy using average model is 0.9819
After 4000 training step(s), validation accuracy using averag

# 5.4.1 持久化代码实现

In [22]:
import tensorflow as tf
import os

v1 = tf.Variable(tf.constant(1.0, shape=[1]), name="v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name="v2")
result = v1 + v2

init_op = tf.global_variables_initializer()
saver = tf.train.Saver()
model_path = os.path.join("model", "model.ckpt")

with tf.Session() as sess:
    sess.run(init_op)
    saver.save(sess, model_path)

In [ ]:
import tensorflow as tf
import os

v1 = tf.Variable(tf.constant(1.0, shape=[1]), name='v1')
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name='v2')
result = v1 + v2

saver = tf.train.Saver()
model_path = os.path.join("model", "model.ckpt")

with tf.Session() as sess:
    saver.restore(sess, model_path)
    print(sess.run(result))

In [ ]:
import tensorflow as tf
import os

model_path = os.path.join("model", "model.ckpt")
meta_path = model_path + ".meta"
saver = tf.train.import_meta_graph(meta_path)

with tf.Session() as sess:
    saver.restore(sess, model_path)
    print(sess.run(tf.get_default_graph().get_tensor_by_name("add:0")))

In [ ]:
import tensorflow as tf
import os

v1 = tf.Variable(tf.constant(1.0, shape=[1]), name='other-v1')
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name='other-v2')
result = v1 + v2

saver = tf.train.Saver({"v1": v1, "v2": v2})
model_path = os.path.join("model", "model.ckpt")

with tf.Session() as sess:
    saver.restore(sess, model_path)
    print(sess.run(result))

In [27]:
import tensorflow as tf
import os

v = tf.Variable(0, dtype=tf.float32, name="v")

for variable in tf.global_variables():
    print(variable.name)
    
ema = tf.train.ExponentialMovingAverage(0.99)
maintain_averages_op = ema.apply(tf.global_variables())

print("After ema")

for variable in tf.global_variables():
    print(variable.name)
    
saver = tf.train.Saver()
model_path = os.path.join("model", "model_2.ckpt")

with tf.Session() as sess:
    init_op = tf.global_variables_initializer()
    sess.run(init_op)
    
    sess.run(tf.assign(v, 10))
    sess.run(maintain_averages_op)
    saver.save(sess, model_path)
    print(sess.run([v, ema.average(v)]))

Variable:0
Variable_1:0
beta1_power:0
beta2_power:0
Variable/Adam:0
Variable/Adam_1:0
Variable_1/Adam:0
Variable_1/Adam_1:0
Variable_2:0
beta1_power_1:0
beta2_power_1:0
Variable_2/Adam:0
Variable_2/Adam_1:0
Variable_3:0
Variable_4:0
Variable_5:0
Variable_6:0
Variable_7:0
Variable_8:0
Variable_9:0
Variable/ExponentialMovingAverage:0
Variable_1/ExponentialMovingAverage:0
Variable_2/ExponentialMovingAverage:0
Variable_3/ExponentialMovingAverage:0
Variable_4/ExponentialMovingAverage:0
Variable_5/ExponentialMovingAverage:0
Variable_6/ExponentialMovingAverage:0
Variable_7/ExponentialMovingAverage:0
Variable_8/ExponentialMovingAverage:0
Variable_10:0
Variable_11:0
Variable_12:0
Variable_13:0
Variable_14:0
Variable/ExponentialMovingAverage_1:0
Variable_1/ExponentialMovingAverage_1:0
Variable_2/ExponentialMovingAverage_1:0
Variable_3/ExponentialMovingAverage_1:0
Variable_4/ExponentialMovingAverage_1:0
Variable_5/ExponentialMovingAverage_1:0
Variable_6/ExponentialMovingAverage_1:0
Variable_7/Exp

TypeError: The variables must be half, float, or double: Variable_9:0

In [28]:
import tensorflow as tf
import os

v = tf.Variable(0, dtype=tf.float32, name="v")
    
saver = tf.train.Saver({"v/ExponentialMovingAverage": v})
model_path = os.path.join("model", "model_2.ckpt")

with tf.Session() as sess:
    saver.restore(sess, model_path)
    print(sess.run(v))


INFO:tensorflow:Restoring parameters from model/model_2.ckpt
0.0999999


In [29]:
import tensorflow as tf
import os

v = tf.Variable(0, dtype=tf.float32, name="v")
ema = tf.train.ExponentialMovingAverage(0.99)

print(ema.variables_to_restore())
    
saver = tf.train.Saver(ema.variables_to_restore())
model_path = os.path.join("model", "model_2.ckpt")

with tf.Session() as sess:
    saver.restore(sess, model_path)
    print(sess.run(v))

{'Variable/ExponentialMovingAverage': <tf.Variable 'Variable/ExponentialMovingAverage:0' shape=(2, 3) dtype=float32_ref>, 'Variable_1/Adam_1/ExponentialMovingAverage': <tf.Variable 'Variable_1/Adam_1/ExponentialMovingAverage:0' shape=(3, 1) dtype=float32_ref>, 'Variable_5/ExponentialMovingAverage_2': <tf.Variable 'Variable_5/ExponentialMovingAverage_2:0' shape=(784, 500) dtype=float32_ref>, 'v_4/ExponentialMovingAverage': <tf.Variable 'v_4:0' shape=() dtype=float32_ref>, 'v/ExponentialMovingAverage': <tf.Variable 'v:0' shape=() dtype=float32_ref>, 'v_2/ExponentialMovingAverage': <tf.Variable 'v_2:0' shape=() dtype=float32_ref>, 'v2_5/ExponentialMovingAverage': <tf.Variable 'v2_5:0' shape=(1,) dtype=float32_ref>, 'Variable_1/Adam_1/ExponentialMovingAverage_3': <tf.Variable 'Variable_1/Adam_1:0' shape=(3, 1) dtype=float32_ref>, 'Variable_4/ExponentialMovingAverage_1': <tf.Variable 'Variable_4/ExponentialMovingAverage_1:0' shape=(500,) dtype=float32_ref>, 'beta1_power_1/ExponentialMovingA

INFO:tensorflow:Restoring parameters from model/model_2.ckpt


NotFoundError: Key v_4/ExponentialMovingAverage not found in checkpoint
	 [[Node: save_9/RestoreV2_124 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_arg_save_9/Const_0_0, save_9/RestoreV2_124/tensor_names, save_9/RestoreV2_124/shape_and_slices)]]

Caused by op 'save_9/RestoreV2_124', defined at:
  File "/Users/colinzuo/anaconda/envs/python35/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/colinzuo/anaconda/envs/python35/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/colinzuo/anaconda/envs/python35/lib/python3.5/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/colinzuo/anaconda/envs/python35/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/colinzuo/anaconda/envs/python35/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 477, i

In [ ]:
import tensorflow as tf
from tensorflow.python.framework import graph_util
import os

v1 = tf.Variable(tf.constant(1.0, shape=[1]), name='v1')
v2 = tf.Variable(tf.constant(2.0, shape=[2]), name='v2')
result = v1 + v2

model_path = os.path.join("model", "combined_model.pb")

init_op = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init_op)
    
    graph_def = tf.get_default_graph().as_graph_def()
    output_graph_def = graph_util.convert_variables_to_constants(sess, graph_def, ['add'])
    
    with tf.gfile.GFile(model_path, "wb") as f:
        f.write(output_graph_def.SerializeToString())

In [ ]:
import tensorflow as tf
from tensorflow.python.platform import gfile
import os

model_path = os.path.join("model", "combined_model.pb")

with tf.Session() as sess:
    with gfile.FastGFile(model_path, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    result = tf.import_graph_def(graph_def, return_elements=['add:0'])
    print(sess.run(result))